In [1]:
#from transformers.utils.fx import symbolic_trace
from transformers import AutoConfig, AutoFeatureExtractor, AutoTokenizer, AutoProcessor, AutoModelForSpeechSeq2Seq
from datasets import load_dataset
import torch
from torch.fx import symbolic_trace
import torch.fx as fx

In [2]:
model_name_or_path = 'openai/whisper-small'
data_dir = 'mozilla-foundation/common_voice_11_0'

In [3]:
# load dataset
print('loading dataset from {}'.format(data_dir))

raw_datasets = load_dataset(data_dir, "zh-CN", split="test", streaming=True)
text_column_name = 'sentence'


# model, tokenizer, feature extractor, processor

model_config = AutoConfig.from_pretrained(
    model_name_or_path,
    #cache_dir=args.cache_dir,
    #revision=args.model_revision,
    #use_auth_token=True if args.use_auth_token else None,
)

model_config.update({"forced_decoder_ids": [], "suppress_tokens": []})


feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_name_or_path,
    #cache_dir=args.cache_dir,
    #revision=args.model_revision,
    #use_auth_token=True if args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    #cache_dir=args.cache_dir,
    #use_fast=model_args.use_fast_tokenizer,
    #revision=model_args.model_revision,
    #use_auth_token=True if model_args.use_auth_token else None,
)

tokenizer.set_prefix_tokens(language='chinese', task='transcribe')


model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_name_or_path,
    config=model_config,
    #cache_dir=args.cache_dir,
    #revision=args.model_revision,
    #use_auth_token=True if args.use_auth_token else None,
)

if model.config.decoder_start_token_id is None:
    raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")
    
processor = AutoProcessor.from_pretrained(model_name_or_path)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language='chinese', task='transcribe')


dataset = raw_datasets

model.eval()


sample = next(iter(dataset))


inputs = processor(
    sample['audio']["array"],
    sampling_rate=feature_extractor.sampling_rate,
    return_attention_mask=True,
    return_tensors="pt")
                
input_features = inputs.input_features
attention_mask = inputs.attention_mask
decoder_input_ids = torch.tensor([model.config.decoder_start_token_id]).reshape(1, -1)

loading dataset from mozilla-foundation/common_voice_11_0


/Users/ujan/opt/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/whisper/modeling_whisper.py:1448: UserWarning: disabled check : 
                attn_weights.size() == (bsz * self.num_heads, tgt_len, src_len) 
            in WhisperAttention.forward
  warnings.warn(
/Users/ujan/opt/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/whisper/modeling_whisper.py:1453: UserWarning: disabled check :
                attn_output.size() == (bsz * self.num_heads, tgt_len, self.head_dim)
            in WhisperAttention.forward
  warnings.warn(
/Users/ujan/opt/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/whisper/modeling_whisper.py:1458: UserWarning: if passing in a tuple for encoder_outputs, wrap it in a BaseModelOutput when return_dict=True
                before passing through model. As :
                    encoder_outputs = BaseModelOutput(
                    last_hidden_state=encoder_outputs[0],
                    hidden_states=

In [4]:
encoder = model.get_encoder()

In [5]:
encoder_concrete_args={
                'output_hidden_states': False,
                'output_attentions': False,
                'head_mask': None,
                'return_dict': True,
              }

decoder_concrete_args={
                'output_hidden_states': False,
                'output_attentions': False,
                'return_dict': True,
                'use_cache': True,
                'decoder_inputs_embeds': None,
              }

In [6]:
symbolic_traced : torch.fx.GraphModule = symbolic_trace(encoder, concrete_args=encoder_concrete_args)

In [7]:
print(symbolic_traced.code)




def forward(self, input_features, attention_mask = None, head_mask_1 = None, output_attentions_1 = None, output_hidden_states_1 = None, return_dict_1 = None):
    _assert_is_none = torch.fx._symbolic_trace._assert_is_none(head_mask_1, 'head_mask has been specialized to have value None but got another value');  head_mask_1 = None
    eq = output_attentions_1 == False;  output_attentions_1 = None
    _assert = torch._assert(eq, 'output_attentions has been specialized to have value False but got another value');  eq = None
    eq_1 = output_hidden_states_1 == False;  output_hidden_states_1 = None
    _assert_1 = torch._assert(eq_1, 'output_hidden_states has been specialized to have value False but got another value');  eq_1 = None
    eq_2 = return_dict_1 == True;  return_dict_1 = None
    _assert_2 = torch._assert(eq_2, 'return_dict has been specialized to have value True but got another value');  eq_2 = None
    conv1 = self.conv1(input_features);  input_features = None
    gelu = to

In [11]:
symbolic_traced : torch.fx.GraphModule = symbolic_trace(model, concrete_args=decoder_concrete_args)

In [12]:
print(symbolic_traced.code)


torch.fx._symbolic_trace.wrap("transformers_models_whisper_modeling_whisper__prepare_decoder_attention_mask")

def forward(self, input_features : typing_Union[torch.FloatTensor,NoneType] = None, attention_mask : typing_Union[torch.LongTensor,NoneType] = None, decoder_input_ids : typing_Union[torch.LongTensor,NoneType] = None, decoder_attention_mask : typing_Union[torch.LongTensor,NoneType] = None, head_mask : typing_Union[torch.Tensor,NoneType] = None, decoder_head_mask : typing_Union[torch.Tensor,NoneType] = None, cross_attn_head_mask : typing_Union[torch.Tensor,NoneType] = None, encoder_outputs : typing_Union[typing_Tuple[typing_Tuple[torch.FloatTensor]],NoneType] = None, past_key_values : typing_Union[typing_Tuple[typing_Tuple[torch.FloatTensor]],NoneType] = None, decoder_inputs_embeds_1 = None, labels : typing_Union[torch.LongTensor,NoneType] = None, use_cache_1 = None, output_attentions_1 = None, output_hidden_states_1 = None, return_dict_1 = None) -> typing_Union[typing_Tuple[to